<a href="https://colab.research.google.com/github/rxymitchy/Colabnotebooks/blob/main/PT_Static_Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import minimize, brute
from scipy.spatial import ConvexHull
import itertools

In [ ]:
# ============================================================================
# 1. PROSPECT THEORY IMPLEMENTATION (with verification)
# ============================================================================

class ProspectTheory:
    """Prospect Theory value and probability weighting functions"""

    def __init__(self, lambd=2.25, alpha=0.88, gamma=0.61, r=0):
        """
        Initialize PT parameters

        Parameters:
        -----------
        lambd : float, loss aversion coefficient (λ > 1)
        alpha : float, diminishing sensitivity (0 < α ≤ 1)
        gamma : float, probability weighting parameter (0 < γ ≤ 1)
        r : float, reference point
        """
        self.lambd = lambd
        self.alpha = alpha
        self.gamma = gamma
        self.r = r

    def probability_weighting(self, p):
        """Prelec probability weighting function"""
        if p <= 0:
            return 0
        elif p >= 1:
            return 1
        else:
            return np.exp(-np.power(-np.log(p), self.gamma))

    def probability_weighting_tk(self, p):
        """Tversky & Kahneman (1992) probability weighting function"""
        if p <= 0:
            return 0
        elif p >= 1:
            return 1
        else:
            return p**self.gamma / (p**self.gamma + (1-p)**self.gamma)**(1/self.gamma)

    def value_function(self, x):
        """PT value function v(x)"""
        if x >= 0:
            return x**self.alpha
        else:
            return -self.lambd * ((-x)**self.alpha)

    def get_value_matrix(self, payoff_matrix):
        """Calculate PT value matrix relative to reference point"""
        value_matrix = np.zeros_like(payoff_matrix)

        for i in range(payoff_matrix.shape[0]):
            for j in range(payoff_matrix.shape[1]):
                # For 2-player game, each player's payoff
                for player in [0, 1]:
                    payoff = payoff_matrix[i, j, player]
                    value_matrix[i, j, player] = self.value_function(payoff - self.r)

        return value_matrix

    def calculate_pt_payoff(self, p, q, payoff_matrix, player):
        """
        Calculate PT expected payoff for a player given mixed strategies

        Parameters:
        -----------
        p : float, probability player 1 plays row 0
        q : float, probability player 2 plays column 0
        payoff_matrix : ndarray, original payoff matrix
        player : int, 0 for row player, 1 for column player
        """
        # Get value matrix
        value_matrix = self.get_value_matrix(payoff_matrix)

        if player == 0:  # Row player
            # Outcomes: (0,0), (0,1), (1,0), (1,1)
            outcomes = [
                value_matrix[0, 0, 0],  # p*q
                value_matrix[0, 1, 0],  # p*(1-q)
                value_matrix[1, 0, 0],  # (1-p)*q
                value_matrix[1, 1, 0]   # (1-p)*(1-q)
            ]
            probabilities = [p*q, p*(1-q), (1-p)*q, (1-p)*(1-q)]
        else:  # Column player
            outcomes = [
                value_matrix[0, 0, 1],
                value_matrix[0, 1, 1],
                value_matrix[1, 0, 1],
                value_matrix[1, 1, 1]
            ]
            probabilities = [p*q, p*(1-q), (1-p)*q, (1-p)*(1-q)]

        # Apply probability weighting
        weighted_outcomes = []
        for prob, outcome in zip(probabilities, outcomes):
            if prob > 0:
                # Decision weight = w(p) (simplified - should be cumulative)
                weighted_outcomes.append(self.probability_weighting(prob) * outcome)

        return np.sum(weighted_outcomes)

In [ ]:
# ============================================================================
# 2. GAME DEFINITIONS (expanded set)
# ============================================================================

def get_all_games():
    """Return dictionary of all games for testing"""

    games = {
        # ============================================================
        # ORIGINAL GAMES
        # ============================================================
        'PrisonersDilemma': {
            'payoffs': np.array([
                [[-1, -1], [-3, 0]],   # Cooperate/Cooperate, Cooperate/Defect
                [[0, -3], [-2, -2]]    # Defect/Cooperate, Defect/Defect
            ]),
            'description': 'Classic Prisoner\'s Dilemma',
            'classical_NE': {
                'pure': [(1, 1)],  # Both defect (index 1 = Defect)
                'mixed': None
            }
        },

        'MatchingPennies': {
            'payoffs': np.array([
                [[1, -1], [-1, 1]],   # Heads/Heads, Heads/Tails
                [[-1, 1], [1, -1]]    # Tails/Heads, Tails/Tails
            ]),
            'description': 'Zero-sum matching pennies',
            'classical_NE': {
                'pure': [],
                'mixed': (0.5, 0.5)
            }
        },

        'OchsGame_Correct': {
            'payoffs': np.array([
                [[0, 0], [5, 8]],    # Top/Left, Top/Right
                [[8, 5], [0, 0]]     # Bottom/Left, Bottom/Right
            ]),
            'description': 'Ochs game that causes PT-NE pathology',
            'classical_NE': {
                'pure': [],
                'mixed': (0.2, 0.5)  # p=0.2, q=0.5
            }
        },

        'CrawfordGame': {
            'payoffs': np.array([
                [[0, 0], [5, 1]],    # Top/Left, Top/Right
                [[1, 5], [4, 4]]     # Bottom/Left, Bottom/Right
            ]),
            'description': 'Crawford game with convex preferences',
            'classical_NE': {
                'pure': [(0, 1), (1, 0)],  # (Top,Right) and (Bottom,Left)
                'mixed': (0.833, 0.833)    # p=5/6 ≈ 0.833
            }
        },

        'BattleOfSexes': {
            'payoffs': np.array([
                [[2, 1], [0, 0]],    # Opera/Opera, Opera/Football
                [[0, 0], [1, 2]]     # Football/Opera, Football/Football
            ]),
            'description': 'Classic Battle of the Sexes',
            'classical_NE': {
                'pure': [(0, 0), (1, 1)],
                'mixed': (2/3, 1/3)  # p=2/3, q=1/3
            }
        },

        # ============================================================
        # NEW GAMES FOR COMPREHENSIVE TESTING
        # ============================================================
        'StagHunt': {
            'payoffs': np.array([
                [[4, 4], [0, 3]],    # Stag/Stag, Stag/Hare
                [[3, 0], [2, 2]]     # Hare/Stag, Hare/Hare
            ]),
            'description': 'Stag Hunt - coordination with risk',
            'classical_NE': {
                'pure': [(0, 0), (1, 1)],  # Both Stag or both Hare
                'mixed': (0.5, 0.5)
            }
        },

        'HawkDove': {
            'payoffs': np.array([
                [[0, 0], [4, 1]],    # Hawk/Hawk, Hawk/Dove
                [[1, 4], [2, 2]]     # Dove/Hawk, Dove/Dove
            ]),
            'description': 'Hawk-Dove / Chicken game',
            'classical_NE': {
                'pure': [(0, 1), (1, 0)],  # One Hawk, one Dove
                'mixed': (0.5, 0.5)
            }
        },

        'AsymmetricBoS': {
            'payoffs': np.array([
                [[3, 2], [0, 0]],    # Opera/Opera, Opera/Football
                [[0, 0], [2, 3]]     # Football/Opera, Football/Football
            ]),
            'description': 'Asymmetric Battle of Sexes',
            'classical_NE': {
                'pure': [(0, 0), (1, 1)],
                'mixed': (0.6, 0.4)  # p=0.6 Opera, q=0.4 Opera
            }
        },

        'PathologicalGame': {
            'payoffs': np.array([
                [[5, 5], [0, 8]],    # Top/Left, Top/Right
                [[8, 0], [1, 1]]     # Bottom/Left, Bottom/Right
            ]),
            'description': 'Designed to potentially break PT-NE',
            'classical_NE': {
                'pure': [(0, 0), (1, 1)],
                'mixed': (0.5, 0.5)
            }
        },

        'RiskShiftGame': {
            'payoffs': np.array([
                [[2, 2], [0, 3]],    # Safe/Safe, Safe/Risky
                [[3, 0], [1, 1]]     # Risky/Safe, Risky/Risky
            ]),
            'description': 'Game sensitive to reference point',
            'classical_NE': {
                'pure': [(0, 0), (1, 1)],
                'mixed': (0.5, 0.5)
            }
        },

        # ============================================================
        # ADDITIONAL TEST CASES
        # ============================================================
        'PureCoordination': {
            'payoffs': np.array([
                [[1, 1], [0, 0]],    # A/A, A/B
                [[0, 0], [1, 1]]     # B/A, B/B
            ]),
            'description': 'Pure coordination game',
            'classical_NE': {
                'pure': [(0, 0), (1, 1)],
                'mixed': (0.5, 0.5)
            }
        },

        'Deadlock': {
            'payoffs': np.array([
                [[1, 1], [3, 0]],    # Cooperate/Cooperate, Cooperate/Defect
                [[0, 3], [2, 2]]     # Defect/Cooperate, Defect/Defect
            ]),
            'description': 'Deadlock game (variant of PD)',
            'classical_NE': {
                'pure': [(1, 1)],
                'mixed': None
            }
        }
    }

    return games

In [ ]:
# ============================================================================
# 3. EQUILIBRIUM FINDING FUNCTIONS
# ============================================================================

def find_classical_ne(payoff_matrix):
    """Find classical Nash equilibria analytically for 2x2 games"""
    pure_NE = []
    mixed_NE = None

    # Check pure strategy NE
    for i in [0, 1]:  # Player 1 strategies
        for j in [0, 1]:  # Player 2 strategies
            # Check if i is best response to j for player 1
            payoff_i_j = payoff_matrix[i, j, 0]
            payoff_other_j = payoff_matrix[1-i, j, 0]

            # Check if j is best response to i for player 2
            payoff_i_j2 = payoff_matrix[i, j, 1]
            payoff_i_other = payoff_matrix[i, 1-j, 1]

            if payoff_i_j >= payoff_other_j and payoff_i_j2 >= payoff_i_other:
                pure_NE.append((i, j))

    # Find mixed strategy NE (if exists)
    # For 2x2 games: p such that player 2 is indifferent
    A = payoff_matrix[0, 0, 1] - payoff_matrix[0, 1, 1] - payoff_matrix[1, 0, 1] + payoff_matrix[1, 1, 1]
    B = payoff_matrix[0, 1, 1] - payoff_matrix[1, 1, 1]

    if abs(A) > 1e-10:
        p_mixed = B / A
        # q such that player 1 is indifferent
        C = payoff_matrix[0, 0, 0] - payoff_matrix[1, 0, 0] - payoff_matrix[0, 1, 0] + payoff_matrix[1, 1, 0]
        D = payoff_matrix[1, 0, 0] - payoff_matrix[1, 1, 0]

        if abs(C) > 1e-10:
            q_mixed = D / C

            # Check if valid probabilities
            if 0 <= p_mixed <= 1 and 0 <= q_mixed <= 1:
                # Check if it's not already a pure strategy
                if not (abs(p_mixed) < 1e-10 or abs(p_mixed-1) < 1e-10 or
                        abs(q_mixed) < 1e-10 or abs(q_mixed-1) < 1e-10):
                    mixed_NE = (p_mixed, q_mixed)

    return pure_NE, mixed_NE

def find_pt_nash_equilibrium(payoff_matrix, pt, grid_points=21):
    """
    Find Prospect Theory Nash Equilibrium using grid search

    Parameters:
    -----------
    payoff_matrix : ndarray, game payoff matrix
    pt : ProspectTheory object
    grid_points : int, number of grid points for search

    Returns:
    --------
    tuple or None: (p, q) if PT-NE found, None otherwise
    """
    best_eq = None
    min_deviation = float('inf')

    # Create grid
    p_values = np.linspace(0, 1, grid_points)
    q_values = np.linspace(0, 1, grid_points)

    for p in p_values:
        for q in q_values:
            # Calculate best responses
            br1 = find_pt_best_response(q, payoff_matrix, pt, player=0)
            br2 = find_pt_best_response(p, payoff_matrix, pt, player=1)

            # Check deviation from equilibrium
            deviation = abs(p - br1) + abs(q - br2)

            if deviation < min_deviation:
                min_deviation = deviation
                best_eq = (p, q, br1, br2, deviation)

    # If deviation is small enough, return as equilibrium
    if min_deviation < 0.05:  # 5% tolerance
        p, q, _, _, _ = best_eq
        return (p, q)
    else:
        return None

def find_pt_equilibrium_in_beliefs(payoff_matrix, pt, grid_points=51):
    """
    Find Prospect Theory Equilibrium-in-Beliefs

    Parameters:
    -----------
    payoff_matrix : ndarray, game payoff matrix
    pt : ProspectTheory object
    grid_points : int, number of grid points for search

    Returns:
    --------
    tuple or None: (p, q) if PT-EB found, None otherwise
    """
    # PT-EB: Each player optimizes given their beliefs about other's strategy
    # We look for fixed point in belief space

    best_eq = None
    min_deviation = float('inf')

    # Create belief grid
    p_beliefs = np.linspace(0, 1, grid_points)
    q_beliefs = np.linspace(0, 1, grid_points)

    for p_belief in p_beliefs:  # Player 2's belief about player 1
        for q_belief in q_beliefs:  # Player 1's belief about player 2
            # Given beliefs, players choose optimal strategies
            p_strat = find_pt_best_response(q_belief, payoff_matrix, pt, player=0)
            q_strat = find_pt_best_response(p_belief, payoff_matrix, pt, player=1)

            # Check if beliefs match chosen strategies
            deviation = abs(p_belief - p_strat) + abs(q_belief - q_strat)

            if deviation < min_deviation:
                min_deviation = deviation
                best_eq = (p_strat, q_strat, p_belief, q_belief, deviation)

    # Return if beliefs are consistent with strategies
    if min_deviation < 0.05:  # 5% tolerance
        p_strat, q_strat, _, _, _ = best_eq
        return (p_strat, q_strat)
    else:
        return None

def find_pt_best_response(opponent_strategy, payoff_matrix, pt, player=0):
    """
    Find best response given opponent's strategy

    Parameters:
    -----------
    opponent_strategy : float, probability opponent plays strategy 0
    payoff_matrix : ndarray, game payoff matrix
    pt : ProspectTheory object
    player : int, 0 for row player, 1 for column player

    Returns:
    --------
    float: optimal probability to play strategy 0
    """
    best_payoff = -float('inf')
    best_response = 0.5  # Default

    # Test discrete strategies first (simplified)
    for p_test in [0.0, 0.25, 0.5, 0.75, 1.0]:
        if player == 0:
            payoff = pt.calculate_pt_payoff(p_test, opponent_strategy, payoff_matrix, 0)
        else:
            payoff = pt.calculate_pt_payoff(opponent_strategy, p_test, payoff_matrix, 1)

        if payoff > best_payoff:
            best_payoff = payoff
            best_response = p_test

    return best_response

In [ ]:
# ============================================================================
# 4. IMPLEMENTATION VERIFICATION FUNCTIONS
# ============================================================================

def verify_pt_implementation():
    """Verify PT implementation matches theoretical properties"""

    checks = []

    # Create PT object with standard parameters
    pt = ProspectTheory(lambd=2.25, alpha=0.88, gamma=0.61, r=0)

    # 1. Check probability weighting shape (inverse S)
    p_low = 0.1
    p_high = 0.9
    w_low = pt.probability_weighting(p_low)
    w_high = pt.probability_weighting(p_high)

    checks.append({
        'test': 'Probability Weighting (low p)',
        'result': f'w({p_low}) = {w_low:.3f} > {p_low}',
        'passed': w_low > p_low,
        'required': f'w(p) > p for small p (inverse S-shape)'
    })

    checks.append({
        'test': 'Probability Weighting (high p)',
        'result': f'w({p_high}) = {w_high:.3f} < {p_high}',
        'passed': w_high < p_high,
        'required': f'w(p) < p for large p (inverse S-shape)'
    })

    # 2. Check loss aversion
    gain = 100
    loss = -100
    value_gain = pt.value_function(gain)
    value_loss = pt.value_function(loss)
    loss_aversion_ratio = abs(value_loss / value_gain)

    checks.append({
        'test': 'Loss Aversion',
        'result': f'λ_effective = {loss_aversion_ratio:.2f} (λ={pt.lambd})',
        'passed': loss_aversion_ratio > 1.5,  # Should be > 1 due to loss aversion
        'required': 'Losses weighted more heavily than gains'
    })

    # 3. Check diminishing sensitivity
    gain_small = 10
    gain_large = 100
    value_gain_small = pt.value_function(gain_small)
    value_gain_large = pt.value_function(gain_large)

    # Marginal value should decrease
    marginal_small = value_gain_small / gain_small
    marginal_large = (value_gain_large - value_gain_small) / (gain_large - gain_small)

    checks.append({
        'test': 'Diminishing Sensitivity (gains)',
        'result': f'Marginal value: {marginal_small:.3f} > {marginal_large:.3f}',
        'passed': marginal_small > marginal_large,
        'required': 'Decreasing marginal utility for gains'
    })

    # 4. Check reference point
    pt_with_ref = ProspectTheory(lambd=2.25, alpha=0.88, gamma=0.61, r=50)
    payoff = 60  # 10 units above reference
    value = pt_with_ref.value_function(payoff - pt_with_ref.r)

    checks.append({
        'test': 'Reference Point',
        'result': f'Payoff {payoff}, ref {pt_with_ref.r}, value = {value:.2f}',
        'passed': value > 0,  # Should be positive (gain)
        'required': 'Values calculated relative to reference point'
    })

    return checks, pt

def fine_search_ochs_game():
    """Ultra-fine grid search for PT-EB in Ochs game"""

    # Ochs game as defined
    ochs_game = np.array([
        [[0, 0], [5, 8]],
        [[8, 5], [0, 0]]
    ])

    # Parameters from thesis claim for Ochs game
    pt_params = [
        {'lambd': 0, 'alpha': 1.0, 'gamma': 1.0, 'r': 1.0, 'name': 'Thesis claim'},
        {'lambd': 0, 'alpha': 1.0, 'gamma': 0.61, 'r': 1.0, 'name': 'With prob weighting'},
        {'lambd': 1.0, 'alpha': 1.0, 'gamma': 1.0, 'r': 1.0, 'name': 'With loss aversion'},
    ]

    results = []

    for params in pt_params:
        pt = ProspectTheory(**{k: v for k, v in params.items() if k != 'name'})

        # Ultra-fine grid around thesis claim: (0.5, 0.05)
        p_range = np.linspace(0.4, 0.6, 201)  # 0.001 steps
        q_range = np.linspace(0.0, 0.15, 151) # 0.001 steps

        best_eq = None
        min_deviation = float('inf')

        for p in p_range:
            for q in q_range:
                # Calculate PT best responses
                br1 = find_pt_best_response(q, ochs_game, pt, player=0)
                br2 = find_pt_best_response(p, ochs_game, pt, player=1)

                # Check if (p, br1) and (br2, q) are close
                deviation = abs(p - br1) + abs(q - br2)

                if deviation < min_deviation:
                    min_deviation = deviation
                    best_eq = (p, q, br1, br2, deviation)

        if best_eq:
            p, q, br1, br2, dev = best_eq
            results.append({
                'params': params['name'],
                'p': p,
                'q': q,
                'br1': br1,
                'br2': br2,
                'deviation': dev,
                'close_to_claim': abs(p-0.5) < 0.05 and abs(q-0.05) < 0.05
            })

    return results

In [ ]:
# ============================================================================
# 5. MAIN ANALYSIS FUNCTION
# ============================================================================

def analyze_game(game_name, game_data, pt_params):
    """
    Analyze a single game with given PT parameters

    Parameters:
    -----------
    game_name : str, name of the game
    game_data : dict, game definition
    pt_params : dict, PT parameters

    Returns:
    --------
    dict: analysis results
    """
    payoff_matrix = game_data['payoffs']
    pt = ProspectTheory(**pt_params)

    # Find classical NE
    pure_ne, mixed_ne = find_classical_ne(payoff_matrix)

    # Find PT-NE
    pt_ne = find_pt_nash_equilibrium(payoff_matrix, pt, grid_points=31)

    # Find PT-EB
    pt_eb = find_pt_equilibrium_in_beliefs(payoff_matrix, pt, grid_points=41)

    # Compare with classical
    pt_vs_classical = "Unknown"
    if mixed_ne and pt_ne:
        p_class, q_class = mixed_ne
        p_pt, q_pt = pt_ne
        delta = abs(p_class - p_pt) + abs(q_class - q_pt)
        pt_vs_classical = "Similar" if delta < 0.1 else "Different"

    return {
        'game': game_name,
        'description': game_data['description'],
        'classical_pure': pure_ne,
        'classical_mixed': mixed_ne,
        'pt_ne': pt_ne,
        'pt_eb': pt_eb,
        'pt_vs_classical': pt_vs_classical,
        'pt_params': pt_params.copy()
    }

def run_comprehensive_analysis():
    """Run comprehensive analysis on all games"""

    # Get all games
    games = get_all_games()

    # Define parameter sets to test
    parameter_sets = [
        # Baseline (standard PT from literature)
        {'lambd': 2.25, 'alpha': 0.88, 'gamma': 0.61, 'r': 0, 'name': 'Baseline'},

        # From thesis examples
        {'lambd': 2.25, 'alpha': 2.0, 'gamma': 1.0, 'r': 0, 'name': 'PrisonersDilemma'},
        {'lambd': 0, 'alpha': 2.0, 'gamma': 1.0, 'r': 0, 'name': 'MatchingPennies'},
        {'lambd': 0, 'alpha': 2.0, 'gamma': 1.0, 'r': 1, 'name': 'OchsGame'},
        {'lambd': 0, 'alpha': 2.0, 'gamma': 1.0, 'r': 0, 'name': 'CrawfordGame'},
        {'lambd': 1.0, 'alpha': 2.0, 'gamma': 1.0, 'r': 0, 'name': 'BattleOfSexes'},

        # Additional tests
        {'lambd': 0, 'alpha': 1.0, 'gamma': 1.0, 'r': 0, 'name': 'NoLossNoWeight'},
        {'lambd': 1.5, 'alpha': 0.88, 'gamma': 0.61, 'r': 0, 'name': 'ModerateLoss'},
        {'lambd': 2.25, 'alpha': 0.88, 'gamma': 0.8, 'r': 0, 'name': 'WeakWeighting'},
        {'lambd': 2.25, 'alpha': 0.88, 'gamma': 0.4, 'r': 0, 'name': 'StrongWeighting'},
        {'lambd': 2.25, 'alpha': 0.88, 'gamma': 0.61, 'r': 1.0, 'name': 'RefPoint1'},
    ]

    # Game-parameter combinations to test (focused)
    test_combinations = [
        # Thesis core examples
        ('PrisonersDilemma', 'PrisonersDilemma'),
        ('MatchingPennies', 'MatchingPennies'),
        ('OchsGame_Correct', 'OchsGame'),
        ('CrawfordGame', 'CrawfordGame'),
        ('BattleOfSexes', 'BattleOfSexes'),

        # Additional tests
        ('StagHunt', 'Baseline'),
        ('HawkDove', 'Baseline'),
        ('AsymmetricBoS', 'Baseline'),
        ('PathologicalGame', 'Baseline'),
    ]

    results = []

    for game_name, param_name in test_combinations:
        # Find parameter set
        params = next(p for p in parameter_sets if p['name'] == param_name)
        param_dict = {k: v for k, v in params.items() if k != 'name'}

        # Analyze game
        result = analyze_game(game_name, games[game_name], param_dict)
        results.append(result)

    return results

In [ ]:
# ============================================================================
# 6. THESIS CLAIM VERIFICATION
# ============================================================================

def verify_thesis_claims(analysis_results):
    """Verify thesis claims based on analysis results"""

    claims = {
        'pt_ne_pathology': {
            'description': 'PT-NE may fail to exist (pathology)',
            'games': [],
            'supported': False
        },
        'pt_eb_when_ne_fails': {
            'description': 'PT-EB exists when PT-NE fails',
            'games': [],
            'supported': False
        },
        'pure_ne_identical': {
            'description': 'Pure strategy NE are identical under PT',
            'games': [],
            'supported': False
        },
        'mixed_ne_different': {
            'description': 'PT changes mixed strategy predictions',
            'games': [],
            'supported': False
        }
    }

    # Analyze each result
    for result in analysis_results:
        game_name = result['game']

        # Claim 1: PT-NE pathology
        if result['pt_ne'] is None:
            claims['pt_ne_pathology']['games'].append(game_name)
            claims['pt_ne_pathology']['supported'] = True

        # Claim 2: PT-EB when PT-NE fails
        if result['pt_ne'] is None and result['pt_eb'] is not None:
            claims['pt_eb_when_ne_fails']['games'].append(game_name)
            claims['pt_eb_when_ne_fails']['supported'] = True

        # Claim 3: Pure NE identical
        # Check if classical pure NE exist and PT preserves them
        if result['classical_pure']:
            # For each pure NE, check if it's stable under PT
            # (Simplified: assume pure strategies are preserved if PT-NE includes them)
            if result['pt_ne']:
                p_pt, q_pt = result['pt_ne']
                # Check if PT-NE is close to any pure strategy
                for pure_ne in result['classical_pure']:
                    p_pure, q_pure = pure_ne
                    if (abs(p_pt - p_pure) < 0.1 and abs(q_pt - q_pure) < 0.1):
                        claims['pure_ne_identical']['games'].append(game_name)
                        claims['pure_ne_identical']['supported'] = True
                        break

        # Claim 4: Mixed NE different
        if result['classical_mixed'] and result['pt_ne']:
            p_class, q_class = result['classical_mixed']
            p_pt, q_pt = result['pt_ne']
            delta = abs(p_class - p_pt) + abs(q_class - q_pt)
            if delta > 0.1:  # Significant difference
                claims['mixed_ne_different']['games'].append(game_name)
                claims['mixed_ne_different']['supported'] = True

    return claims

In [ ]:
# ============================================================================
# 7. MAIN EXECUTION
# ============================================================================

def main():
    """Main execution function"""

    print("=" * 80)
    print("COMPREHENSIVE VERIFICATION OF LECLERC'S THESIS")
    print("Prospect Theory Equilibrium in Games")
    print("=" * 80)

    # ========================================================================
    # PART 1: IMPLEMENTATION VERIFICATION
    # ========================================================================
    print("\n" + "=" * 80)
    print("PART 1: IMPLEMENTATION VERIFICATION")
    print("=" * 80)

    checks, pt = verify_pt_implementation()

    print("\nPT Implementation Checks:")
    print("-" * 40)
    for check in checks:
        status = "✓" if check['passed'] else "✗"
        print(f"{status} {check['test']:35} {check['result']}")
        if not check['passed']:
            print(f"  Required: {check['required']}")

    # ========================================================================
    # PART 2: FINE SEARCH FOR OCHS GAME PT-EB
    # ========================================================================
    print("\n" + "=" * 80)
    print("PART 2: FINE SEARCH FOR OCHS GAME PT-EB")
    print("=" * 80)

    ochs_results = fine_search_ochs_game()
    print("\nFine Grid Search Results for Ochs Game:")
    print("-" * 50)
    for res in ochs_results:
        status = "✓" if res['close_to_claim'] else "○"
        print(f"{status} {res['params']:25} p={res['p']:.3f}, q={res['q']:.3f}")
        print(f"  Best responses: br1={res['br1']:.3f}, br2={res['br2']:.3f}")
        print(f"  Deviation: {res['deviation']:.6f}")

    # ========================================================================
    # PART 3: COMPREHENSIVE GAME ANALYSIS
    # ========================================================================
    print("\n" + "=" * 80)
    print("PART 3: COMPREHENSIVE GAME ANALYSIS")
    print("=" * 80)

    analysis_results = run_comprehensive_analysis()

    print("\nDetailed Game Analysis:")
    print("-" * 80)

    for result in analysis_results:
        print(f"\nGAME: {result['game']}")
        print(f"Description: {result['description']}")
        print(f"PT Parameters: λ={result['pt_params'].get('lambd', 'N/A')}, "
              f"α={result['pt_params'].get('alpha', 'N/A')}, "
              f"γ={result['pt_params'].get('gamma', 'N/A')}, "
              f"r={result['pt_params'].get('r', 'N/A')}")

        print(f"Classical NE:")
        if result['classical_pure']:
            print(f"  Pure: {result['classical_pure']}")
        if result['classical_mixed']:
            p, q = result['classical_mixed']
            print(f"  Mixed: p={p:.3f}, q={q:.3f}")

        print(f"Prospect Theory:")
        if result['pt_ne']:
            p, q = result['pt_ne']
            print(f"  PT-NE: p={p:.3f}, q={q:.3f}")
            if result['classical_mixed']:
                p_class, q_class = result['classical_mixed']
                delta_p = abs(p_class - p)
                delta_q = abs(q_class - q)
                print(f"    vs Classical: Δp={delta_p:.3f}, Δq={delta_q:.3f}")
        else:
            print(f"  PT-NE: NOT FOUND (pathology)")

        if result['pt_eb']:
            p, q = result['pt_eb']
            print(f"  PT-EB: p={p:.3f}, q={q:.3f}")
        else:
            print(f"  PT-EB: Not found")

    # ========================================================================
    # PART 4: THESIS CLAIM VERIFICATION
    # ========================================================================
    print("\n" + "=" * 80)
    print("PART 4: THESIS CLAIM VERIFICATION")
    print("=" * 80)

    claims = verify_thesis_claims(analysis_results)

    print("\nTHESIS CLAIM ASSESSMENT:")
    print("-" * 60)

    supported_count = 0
    total_claims = len(claims)

    for claim_name, claim_data in claims.items():
        status = "✓" if claim_data['supported'] else "✗"
        if claim_data['supported']:
            supported_count += 1

        print(f"\n{status} {claim_data['description']}")
        if claim_data['games']:
            games_str = ", ".join(claim_data['games'])
            print(f"  Supported by: {games_str}")
        else:
            print(f"  Not supported by tested games")

    # ========================================================================
    # PART 5: SUMMARY AND CONCLUSION
    # ========================================================================
    print("\n" + "=" * 80)
    print("PART 5: SUMMARY AND CONCLUSION")
    print("=" * 80)

    support_rate = supported_count / total_claims * 100

    print(f"\nOVERALL ASSESSMENT:")
    print(f"  Claims supported: {supported_count}/{total_claims} ({support_rate:.1f}%)")

    if support_rate >= 75:
        conclusion = "Thesis claims are LARGELY SUPPORTED"
    elif support_rate >= 50:
        conclusion = "Thesis claims are PARTIALLY SUPPORTED"
    else:
        conclusion = "Thesis claims are LARGELY UNSUPPORTED"

    print(f"  CONCLUSION: {conclusion}")

    print(f"\nKEY FINDINGS:")
    print(f"  1. PT implementation verified: {sum(c['passed'] for c in checks)}/{len(checks)} checks passed")
    print(f"  2. Ochs game PT-EB search: Found candidates close to thesis claim" if
          any(r['close_to_claim'] for r in ochs_results) else
          "  2. Ochs game PT-EB search: No exact match found")

    # Specific findings
    print(f"\nSPECIFIC RESULTS:")
    for result in analysis_results:
        if result['game'] == 'OchsGame_Correct' and result['pt_ne'] is None:
            print(f"  ✓ Ochs Game shows PT-NE failure (as thesis claims)")
        if result['game'] == 'CrawfordGame' and result['pt_vs_classical'] == 'Different':
            print(f"  ✓ Crawford Game shows PT changes predictions")
        if result['game'] == 'BattleOfSexes' and result['pt_vs_classical'] == 'Different':
            print(f"  ✓ Battle of Sexes shows PT changes predictions")

    print("\n" + "=" * 80)
    print("VERIFICATION COMPLETE")
    print("=" * 80)

In [ ]:
# ============================================================================
# ADVANCED DIAGNOSTICS SECTION
# ============================================================================

def run_advanced_diagnostics():
    """Advanced diagnostics for PT equilibrium analysis"""

    print("\n" + "="*80)
    print("ADVANCED DIAGNOSTICS")
    print("="*80)

    # Import needed functions (they should be in same file)
    from scipy.optimize import fsolve

    def debug_best_response(game_name, payoff_matrix, pt):
        """Debug best response functions"""
        print(f"\nDebugging {game_name}:")
        print(f"PT params: λ={pt.lambd}, α={pt.alpha}, γ={pt.gamma}, r={pt.r}")

        # Test BR at key points
        test_points = [0.0, 0.25, 0.5, 0.75, 1.0]
        print("\nPlayer 1 BR(q):")
        for q in test_points:
            br = find_pt_best_response(q, payoff_matrix, pt, 0)
            print(f"  BR({q:.2f}) = {br:.3f}")

        print("\nPlayer 2 BR(p):")
        for p in test_points:
            br = find_pt_best_response(p, payoff_matrix, pt, 1)
            print(f"  BR({p:.2f}) = {br:.3f}")

    def analyze_alpha_effect():
        """Analyze effect of α parameter"""
        print("\n" + "-"*60)
        print("ANALYZING α PARAMETER EFFECT")
        print("-"*60)

        # Matching Pennies game
        mp_game = np.array([
            [[1, -1], [-1, 1]],
            [[-1, 1], [1, -1]]
        ])

        print("\nMatching Pennies with different α (λ=0, γ=1.0, r=0):")
        print("α-value | PT-NE exists? | Notes")
        print("-" * 50)

        for alpha in [0.5, 0.88, 1.0, 1.5, 2.0]:
            pt = ProspectTheory(lambd=0, alpha=alpha, gamma=1.0, r=0)
            pt_ne = find_pt_nash_equilibrium(mp_game, pt, grid_points=21)

            exists = "YES" if pt_ne else "NO"
            note = "Standard PT" if alpha == 0.88 else "Thesis" if alpha == 2.0 else ""

            print(f"α={alpha:.2f} | {exists:11} | {note}")

    # Get games
    games = get_all_games()

    # Run diagnostics
    print("\n1. MATCHING PENNIES PATHOLOGY ANALYSIS")
    mp_pt = ProspectTheory(lambd=0, alpha=2.0, gamma=1.0, r=0)
    debug_best_response('MatchingPennies', games['MatchingPennies']['payoffs'], mp_pt)

    print("\n2. OCHS GAME ANALYSIS")
    ochs_pt = ProspectTheory(lambd=0, alpha=2.0, gamma=1.0, r=1)
    debug_best_response('OchsGame', games['OchsGame_Correct']['payoffs'], ochs_pt)

    # Analyze α parameter
    analyze_alpha_effect()

    # Key insights
    print("\n" + "="*80)
    print("KEY THEORETICAL INSIGHTS")
    print("="*80)

    insights = [
        "1. α=2.0 creates CONVEX value function (unlike standard PT α=0.88)",
        "2. Convex preferences can lead to equilibrium non-existence",
        "3. Matching Pennies pathology explained by convex value function",
        "4. Thesis results are specific to α=2.0 parameterization",
        "5. With standard α=0.88, PT-NE might exist in more games"
    ]

    for insight in insights:
        print(f"• {insight}")

In [ ]:
# ============================================================================
# 8. RUN THE ANALYSIS
# ============================================================================

if __name__ == "__main__":
    main()

    # After displaying results
    print("\n" + "="*80)
    run_diag = input("Run advanced diagnostics? (y/n): ").strip().lower()
    if run_diag == 'y':
        run_advanced_diagnostics()

COMPREHENSIVE VERIFICATION OF LECLERC'S THESIS
Prospect Theory Equilibrium in Games

PART 1: IMPLEMENTATION VERIFICATION

PT Implementation Checks:
----------------------------------------
✓ Probability Weighting (low p)       w(0.1) = 0.190 > 0.1
✓ Probability Weighting (high p)      w(0.9) = 0.776 < 0.9
✓ Loss Aversion                       λ_effective = 2.25 (λ=2.25)
✓ Diminishing Sensitivity (gains)     Marginal value: 0.759 > 0.555
✓ Reference Point                     Payoff 60, ref 50, value = 7.59

PART 2: FINE SEARCH FOR OCHS GAME PT-EB

Fine Grid Search Results for Ochs Game:
--------------------------------------------------
○ Thesis claim              p=0.600, q=0.000
  Best responses: br1=1.000, br2=0.000
  Deviation: 0.400000
○ With prob weighting       p=0.600, q=0.150
  Best responses: br1=0.750, br2=0.250
  Deviation: 0.250000
○ With loss aversion        p=0.600, q=0.000
  Best responses: br1=1.000, br2=0.000
  Deviation: 0.400000

PART 3: COMPREHENSIVE GAME ANALYSIS

